In [2]:
import math
from pprint import pprint
from math import log
import numpy as np
import sys
from typing import List, Tuple, Callable, Iterable
from functools import lru_cache, reduce, cmp_to_key
from itertools import (
    starmap,
    accumulate,
    chain,
    islice,
    product,
    tee,
    pairwise,
    combinations,
)
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg, itemgetter
from bisect import bisect_right
import matplotlib.pyplot as plt

import arrays
import trees
import lists
import graphs
import search
import strings
import matrix
import mathematics
from importlib import reload

for m in (arrays, trees, lists, graphs, search, strings, matrix, mathematics):
    reload(m)

import arrays as ar
import trees as tr
import lists as ll
import graphs as g
import strings as s
import matrix as mt
import mathematics as m

print()